In [6]:

%store -r path
import random
import json
import sagemaker
from time import sleep
from datetime import datetime

session = sagemaker.Session()
bucket = session.default_bucket()

ground_truth_upload_path = f"s3://{bucket}/{path}/ground-truth"


def ground_truth_with_id(inference_id):
    random.seed(inference_id)  # to get consistent results
    rand = random.random()
    return {
        "groundTruthData": {
            "data": "1" if rand < 0.7 else "0",  # randomly generate positive labels 70% of the time
            "encoding": "CSV",
        },
        "eventMetadata": {
            "eventId": str(inference_id),
        },
        "eventVersion": "0",
    }


def upload_ground_truth(records, upload_time):
    fake_records = [json.dumps(r) for r in records]
    data_to_upload = "\n".join(fake_records)
    target_s3_uri = f"{ground_truth_upload_path}/{upload_time:%Y/%m/%d/%H/%M%S}.jsonl"
    print(f"Uploading {len(fake_records)} records to", target_s3_uri)
    sagemaker.s3.S3Uploader.upload_string_as_file_body(data_to_upload, target_s3_uri)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [7]:
NUM_GROUND_TRUTH_RECORDS = 100
j = 0
while True:
    fake_records = [ground_truth_with_id(i) for i in range(NUM_GROUND_TRUTH_RECORDS)]
    upload_ground_truth(fake_records, datetime.utcnow())
    j = (j + 1) % 5
    sleep(60 * 60)  # do this once an hour

Uploading 100 records to s3://sagemaker-us-east-1-381492271173/monitor/heart/ground-truth/2024/03/21/00/5203.jsonl
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


KeyboardInterrupt: 